# View and Clean Data
For viewing data, e.g. distribution of values, how many are NaN, etc., and for cleaning data (e.g. replacing missing or invalid values).

## Imports and globals

In [ ]:
!pip install phik
import phik
from phik import resources, report

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
DATASET_FILENAME = "/content/drive/My Drive/ML6140  - Project/Data/enhanced_data.csv"
INITIAL_CLEANED_OUTFILE_NAME = "/content/drive/My Drive/ML6140  - Project/Data/initial_cleaned_data.csv"
CLEANED_OUTFILE_NAME = "/content/drive/My Drive/ML6140  - Project/Data/cleaned_data.csv"
FIGURES_FOLDER = "/content/drive/My Drive/ML6140  - Project/Data/Figures/"

pd.set_option('display.max_columns', None)

## Part 1 - First data set without nan rows (4866 count)
## DF1 = Null columns dropped and null rows dropped, no impossible outliers.

### Read in data

In [ ]:
df1 = pd.read_csv(DATASET_FILENAME)

In [ ]:
df1.shape

(5817, 133)

In [ ]:
df1

,incident_name,incident_created_year,incident_created_month,incident_created_day,incident_created_hour,incident_created_minute,incident_latitude,incident_longitude,incident_geohash,LCD_station_id,LCD_station_elevation,LCD_station_distance,GHCN_station_id,GHCN_station_elevation,GHCN_station_distance,sumPrecipitation,dist_sumPrecipitation,maxDryBulbTemperature,dist_maxDryBulbTemperature,minDryBulbTemperature,dist_minDryBulbTemperature,meanDryBulbTemperature,dist_meanDryBulbTemperature,meanDewPointTemperature,dist_meanDewPointTemperature,meanWetBulbTemperature,dist_meanWetBulbTemperature,meanWindSpeed,dist_meanWindSpeed,meanRelativeHumidity,dist_meanRelativeHumidity,minRelativeHumidity,dist_minRelativeHumidity,maxRelativeHumidity,dist_maxRelativeHumidity,SNOW,dist_SNOW,SNWD,dist_SNWD,EVAP,dist_EVAP,FMTM,dist_FMTM,FRGB,dist_FRGB,FRGT,dist_FRGT,FRTH,dist_FRTH,maxWindSpeed,dist_maxWindSpeed,calculate_circular_meanWindDirection,dist_calculate_circular_meanWindDirection,mode_functionWindDirection,dist_mode_functionWindDirection,maxWindGustSpeed,dist_maxWindGustSpeed,mode_functionPresentWeatherType,dist_mode_functionPresentWeatherType,mode_functionSkyConditions,dist_mode_functionSkyConditions,10_sumPrecipitation,10_maxDryBulbTemperature,10_minDryBulbTemperature,10_meanDryBulbTemperature,10_meanDewPointTemperature,10_meanWetBulbTemperature,10_meanWindSpeed,10_meanRelativeHumidity,10_minRelativeHumidity,10_maxRelativeHumidity,10_SNOW,10_SNWD,10_EVAP,10_FMTM,10_FRGB,10_FRGT,10_FRTH,10_maxWindSpeed,10_calculate_circular_meanWindDirection,10_mode_functionWindDirection,10_maxWindGustSpeed,10_mode_functionPresentWeatherType,10_mode_functionSkyConditions,30_sumPrecipitation,30_maxDryBulbTemperature,30_minDryBulbTemperature,30_meanDryBulbTemperature,30_meanDewPointTemperature,30_meanWetBulbTemperature,30_meanWindSpeed,30_meanRelativeHumidity,30_minRelativeHumidity,30_maxRelativeHumidity,30_SNOW,30_SNWD,30_EVAP,30_FMTM,30_FRGB,30_FRGT,30_FRTH,30_maxWindSpeed,30_calculate_circular_meanWindDirection,30_mode_functionWindDirection,30_maxWindGustSpeed,30_mode_functionPresentWeatherType,30_mode_functionSkyConditions,60_sumPrecipitation,60_maxDryBulbTemperature,60_minDryBulbTemperature,60_meanDryBulbTemperature,60_meanDewPointTemperature,60_meanWetBulbTemperature,60_meanWindSpeed,60_meanRelativeHumidity,60_minRelativeHumidity,60_maxRelativeHumidity,60_SNOW,60_SNWD,60_EVAP,60_FMTM,60_FRGB,60_FRGT,60_FRTH,60_maxWindSpeed,60_calculate_circular_meanWindDirection,60_mode_functionWindDirection,60_maxWindGustSpeed,60_mode_functionPresentWeatherType,60_mode_functionSkyConditions,far_hist_avg_acres_burned,near_hist_avg_acres_burned,class_label
0,Stephens Fire,2015,2,24,12,15,41.485000,-121.851000,9r3k50n,24215,1083.0,43.235962,USC00045449,985.4,35.323794,0.0,35.323794,66.02,35.323794,48.02,35.323794,43.375000,43.235962,8.833333,43.235962,31.166667,43.235962,6.039900,35.323794,27.458333,43.235962,11.0,43.235962,51.0,43.235962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,43.235962,5.809858,43.235962,0.0,43.235962,23.0,43.235962,HZ:7 |FU |HZ,43.235962,CLR:00,43.235962,0.011024,65.876,55.508,44.041667,23.608333,35.737500,5.659610,49.112500,24.7,72.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.1,11.502194,0.0,NaN,NaN,CLR:00,0.004856,69.494000,53.762000,44.375856,31.921307,39.020870,5.450823,66.276835,44.366667,83.533333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.333333,19.407850,0.0,NaN,BR:1 ||,CLR:00,0.007283,67.958000,50.594000,41.523426,30.378900,36.800077,4.947498,68.275119,45.700000,84.450000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.000000,12.517703,0.0,NaN,BR:1 ||,CLR:00,39.6260,136.333333,1
1,Ward Fire,2015,4,13,5,30,40.050833,-120.701667,9r45v75,23225,1611.2,86.141091,USC00047195,1042.4,24.487379,0.0,24.487379,69.08,24.487379,57.92,24.487379,50.153846,86.141091,27.500000,86.141091,39.923077,86.141091,6.039900,24.487379,43.000000,86.141091,27.0,86.141091,79.0,86.141091,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0,86.141091,163.864889,86.141091,170.0,86.141091,25.0

### View Column Info

In [ ]:
pd.set_option('display.max_info_columns', 500)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5817 entries, 0 to 5816
Data columns (total 133 columns):
 #    Column                                     Non-Null Count  Dtype  
---   ------                                     --------------  -----  
 0    incident_name                              5817 non-null   object 
 1    incident_created_year                      5817 non-null   int64  
 2    incident_created_month                     5817 non-null   int64  
 3    incident_created_day                       5817 non-null   int64  
 4    incident_created_hour                      5817 non-null   int64  
 5    incident_created_minute                    5817 non-null   int64  
 6    incident_latitude                          5817 non-null   float64
 7    incident_longitude                         5817 non-null   float64
 8    incident_geohash                           5817 non-null   object 
 9    LCD_station_id                             5817 non-null   int64  
 10   LCD_statio

### Check for duplicates

In [ ]:
# Find duplicates
duplicate_rows = df1.duplicated()

# Count duplicates
num_duplicates = duplicate_rows.sum()
print(f"Number of duplicate rows: {num_duplicates}")

# Display duplicates
if num_duplicates > 0:
    print("Duplicate Rows:")
    print(df1[duplicate_rows])


Number of duplicate rows: 0


### Drop all/mostly null columns

In [ ]:
all_mostly_null_cols = ["SNOW", "dist_SNOW", "SNWD", "dist_SNWD", "EVAP", "dist_EVAP",
                 "FMTM", "dist_FMTM", "FRGB", "dist_FRGB", "FRGT", "dist_FRGT",
                 "FRTH", "dist_FRTH", "maxWindGustSpeed", "dist_maxWindGustSpeed",
                 "mode_functionPresentWeatherType",
                 "dist_mode_functionPresentWeatherType", "10_SNOW", "10_SNWD",
                 "10_EVAP", "10_FMTM", "10_FRGB", "10_FRGT", "10_FRTH",
                 "10_maxWindGustSpeed", "10_mode_functionPresentWeatherType",
                 "30_SNOW", "30_SNWD", "30_EVAP", "30_FMTM", "30_FRGB",
                 "30_FRGT", "30_FRTH", "30_maxWindGustSpeed",
                 "30_mode_functionPresentWeatherType", "60_SNOW", "60_SNWD",
                 "60_EVAP", "60_FMTM", "60_FRGB", "60_FRGT", "60_FRTH",
                  "60_maxWindGustSpeed", "60_mode_functionPresentWeatherType", ]
df1_wo_mostly_null_cols = df1.drop(all_mostly_null_cols, axis=1)
df1_wo_mostly_null_cols

,incident_name,incident_created_year,incident_created_month,incident_created_day,incident_created_hour,incident_created_minute,incident_latitude,incident_longitude,incident_geohash,LCD_station_id,LCD_station_elevation,LCD_station_distance,GHCN_station_id,GHCN_station_elevation,GHCN_station_distance,sumPrecipitation,dist_sumPrecipitation,maxDryBulbTemperature,dist_maxDryBulbTemperature,minDryBulbTemperature,dist_minDryBulbTemperature,meanDryBulbTemperature,dist_meanDryBulbTemperature,meanDewPointTemperature,dist_meanDewPointTemperature,meanWetBulbTemperature,dist_meanWetBulbTemperature,meanWindSpeed,dist_meanWindSpeed,meanRelativeHumidity,dist_meanRelativeHumidity,minRelativeHumidity,dist_minRelativeHumidity,maxRelativeHumidity,dist_maxRelativeHumidity,maxWindSpeed,dist_maxWindSpeed,calculate_circular_meanWindDirection,dist_calculate_circular_meanWindDirection,mode_functionWindDirection,dist_mode_functionWindDirection,mode_functionSkyConditions,dist_mode_functionSkyConditions,10_sumPrecipitation,10_maxDryBulbTemperature,10_minDryBulbTemperature,10_meanDryBulbTemperature,10_meanDewPointTemperature,10_meanWetBulbTemperature,10_meanWindSpeed,10_meanRelativeHumidity,10_minRelativeHumidity,10_maxRelativeHumidity,10_maxWindSpeed,10_calculate_circular_meanWindDirection,10_mode_functionWindDirection,10_mode_functionSkyConditions,30_sumPrecipitation,30_maxDryBulbTemperature,30_minDryBulbTemperature,30_meanDryBulbTemperature,30_meanDewPointTemperature,30_meanWetBulbTemperature,30_meanWindSpeed,30_meanRelativeHumidity,30_minRelativeHumidity,30_maxRelativeHumidity,30_maxWindSpeed,30_calculate_circular_meanWindDirection,30_mode_functionWindDirection,30_mode_functionSkyConditions,60_sumPrecipitation,60_maxDryBulbTemperature,60_minDryBulbTemperature,60_meanDryBulbTemperature,60_meanDewPointTemperature,60_meanWetBulbTemperature,60_meanWindSpeed,60_meanRelativeHumidity,60_minRelativeHumidity,60_maxRelativeHumidity,60_maxWindSpeed,60_calculate_circular_meanWindDirection,60_mode_functionWindDirection,60_mode_functionSkyConditions,far_hist_avg_acres_burned,near_hist_avg_acres_burned,class_label
0,Stephens Fire,2015,2,24,12,15,41.485000,-121.851000,9r3k50n,24215,1083.0,43.235962,USC00045449,985.4,35.323794,0.0,35.323794,66.02,35.323794,48.02,35.323794,43.375000,43.235962,8.833333,43.235962,31.166667,43.235962,6.039900,35.323794,27.458333,43.235962,11.0,43.235962,51.0,43.235962,11.0,43.235962,5.809858,43.235962,0.0,43.235962,CLR:00,43.235962,0.011024,65.876,55.508,44.041667,23.608333,35.737500,5.659610,49.112500,24.7,72.1,10.1,11.502194,0.0,CLR:00,0.004856,69.494000,53.762000,44.375856,31.921307,39.020870,5.450823,66.276835,44.366667,83.533333,8.333333,19.407850,0.0,CLR:00,0.007283,67.958000,50.594000,41.523426,30.378900,36.800077,4.947498,68.275119,45.700000,84.450000,8.000000,12.517703,0.0,CLR:00,39.6260,136.333333,1
1,Ward Fire,2015,4,13,5,30,40.050833,-120.701667,9r45v75,23225,1611.2,86.141091,USC00047195,1042.4,24.487379,0.0,24.487379,69.08,24.487379,57.92,24.487379,50.153846,86.141091,27.500000,86.141091,39.923077,86.141091,6.039900,24.487379,43.000000,86.141091,27.0,86.141091,79.0,86.141091,18.0,86.141091,163.864889,86.141091,170.0,86.141091,CLR:00,86.141091,0.000000,68.090,55.490,40.773436,22.318814,33.373392,7.337360,54.611732,36.6,69.8,12.3,138.314704,0.0,CLR:00,0.000000,71.336000,57.302000,47.705004,25.581872,38.145478,6.278513,47.408972,31.700000,64.033333,12.800000,125.826118,0.0,CLR:00,0.017979,70.166000,55.202000,48.141038,23.987898,38.000429,6.189033,44.454664,27.850000,62.950000,11.666667,73.118840,0.0,CLR:00,1444.3735,46.090000,1
2,Highway Fire,2015,4,18,18,12,33.884313,-117.642759,9qh2fbr,53175,162.5,4.007019,USW00053175,162.5,4.007226,0.0,4.007019,83.00,4.007019,38.00,4.007019,63.500000,4.007019,37.916667,4.007019,50.458333,4.007019,4.083333,4.007019,45.166667,4.007019,19.0,4.007019,80.0,4.007019,15.0,4.007019,328.751345,4.007019,0.0,4.007019,CLR:00,4.007019,0.000000,79.100,42.100,61.461923,37.971859,49.296875,4.848910,52.553141,

#### View Updated info without null columns

In [ ]:
print(df1_wo_mostly_null_cols.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5817 entries, 0 to 5816
Data columns (total 88 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   incident_name                              5817 non-null   object 
 1   incident_created_year                      5817 non-null   int64  
 2   incident_created_month                     5817 non-null   int64  
 3   incident_created_day                       5817 non-null   int64  
 4   incident_created_hour                      5817 non-null   int64  
 5   incident_created_minute                    5817 non-null   int64  
 6   incident_latitude                          5817 non-null   float64
 7   incident_longitude                         5817 non-null   float64
 8   incident_geohash                           5817 non-null   object 
 9   LCD_station_id                             5817 non-null   int64  
 10  LCD_station_elevation   

### Drop rows with null values

In [ ]:
df1_wo_nan_rows_cols = df1_wo_mostly_null_cols.dropna()
df1_wo_nan_rows_cols

,incident_name,incident_created_year,incident_created_month,incident_created_day,incident_created_hour,incident_created_minute,incident_latitude,incident_longitude,incident_geohash,LCD_station_id,LCD_station_elevation,LCD_station_distance,GHCN_station_id,GHCN_station_elevation,GHCN_station_distance,sumPrecipitation,dist_sumPrecipitation,maxDryBulbTemperature,dist_maxDryBulbTemperature,minDryBulbTemperature,dist_minDryBulbTemperature,meanDryBulbTemperature,dist_meanDryBulbTemperature,meanDewPointTemperature,dist_meanDewPointTemperature,meanWetBulbTemperature,dist_meanWetBulbTemperature,meanWindSpeed,dist_meanWindSpeed,meanRelativeHumidity,dist_meanRelativeHumidity,minRelativeHumidity,dist_minRelativeHumidity,maxRelativeHumidity,dist_maxRelativeHumidity,maxWindSpeed,dist_maxWindSpeed,calculate_circular_meanWindDirection,dist_calculate_circular_meanWindDirection,mode_functionWindDirection,dist_mode_functionWindDirection,mode_functionSkyConditions,dist_mode_functionSkyConditions,10_sumPrecipitation,10_maxDryBulbTemperature,10_minDryBulbTemperature,10_meanDryBulbTemperature,10_meanDewPointTemperature,10_meanWetBulbTemperature,10_meanWindSpeed,10_meanRelativeHumidity,10_minRelativeHumidity,10_maxRelativeHumidity,10_maxWindSpeed,10_calculate_circular_meanWindDirection,10_mode_functionWindDirection,10_mode_functionSkyConditions,30_sumPrecipitation,30_maxDryBulbTemperature,30_minDryBulbTemperature,30_meanDryBulbTemperature,30_meanDewPointTemperature,30_meanWetBulbTemperature,30_meanWindSpeed,30_meanRelativeHumidity,30_minRelativeHumidity,30_maxRelativeHumidity,30_maxWindSpeed,30_calculate_circular_meanWindDirection,30_mode_functionWindDirection,30_mode_functionSkyConditions,60_sumPrecipitation,60_maxDryBulbTemperature,60_minDryBulbTemperature,60_meanDryBulbTemperature,60_meanDewPointTemperature,60_meanWetBulbTemperature,60_meanWindSpeed,60_meanRelativeHumidity,60_minRelativeHumidity,60_maxRelativeHumidity,60_maxWindSpeed,60_calculate_circular_meanWindDirection,60_mode_functionWindDirection,60_mode_functionSkyConditions,far_hist_avg_acres_burned,near_hist_avg_acres_burned,class_label
0,Stephens Fire,2015,2,24,12,15,41.485000,-121.851000,9r3k50n,24215,1083.0,43.235962,USC00045449,985.4,35.323794,0.0,35.323794,66.02,35.323794,48.02,35.323794,43.375000,43.235962,8.833333,43.235962,31.166667,43.235962,6.039900,35.323794,27.458333,43.235962,11.0,43.235962,51.0,43.235962,11.0,43.235962,5.809858,43.235962,0.0,43.235962,CLR:00,43.235962,0.011024,65.876,55.508,44.041667,23.608333,35.737500,5.659610,49.112500,24.7,72.1,10.1,11.502194,0.0,CLR:00,0.004856,69.494000,53.762000,44.375856,31.921307,39.020870,5.450823,66.276835,44.366667,83.533333,8.333333,19.407850,0.0,CLR:00,0.007283,67.958000,50.594000,41.523426,30.378900,36.800077,4.947498,68.275119,45.700000,84.450000,8.000000,12.517703,0.0,CLR:00,39.6260,136.333333,1
1,Ward Fire,2015,4,13,5,30,40.050833,-120.701667,9r45v75,23225,1611.2,86.141091,USC00047195,1042.4,24.487379,0.0,24.487379,69.08,24.487379,57.92,24.487379,50.153846,86.141091,27.500000,86.141091,39.923077,86.141091,6.039900,24.487379,43.000000,86.141091,27.0,86.141091,79.0,86.141091,18.0,86.141091,163.864889,86.141091,170.0,86.141091,CLR:00,86.141091,0.000000,68.090,55.490,40.773436,22.318814,33.373392,7.337360,54.611732,36.6,69.8,12.3,138.314704,0.0,CLR:00,0.000000,71.336000,57.302000,47.705004,25.581872,38.145478,6.278513,47.408972,31.700000,64.033333,12.800000,125.826118,0.0,CLR:00,0.017979,70.166000,55.202000,48.141038,23.987898,38.000429,6.189033,44.454664,27.850000,62.950000,11.666667,73.118840,0.0,CLR:00,1444.3735,46.090000,1
2,Highway Fire,2015,4,18,18,12,33.884313,-117.642759,9qh2fbr,53175,162.5,4.007019,USW00053175,162.5,4.007226,0.0,4.007019,83.00,4.007019,38.00,4.007019,63.500000,4.007019,37.916667,4.007019,50.458333,4.007019,4.083333,4.007019,45.166667,4.007019,19.0,4.007019,80.0,4.007019,15.0,4.007019,328.751345,4.007019,0.0,4.007019,CLR:00,4.007019,0.000000,79.100,42.100,61.461923,37.971859,49.296875,4.848910,52.553141,

### View info with no null columns or rows

In [ ]:
print(df1_wo_nan_rows_cols.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4886 entries, 0 to 5816
Data columns (total 88 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   incident_name                              4886 non-null   object 
 1   incident_created_year                      4886 non-null   int64  
 2   incident_created_month                     4886 non-null   int64  
 3   incident_created_day                       4886 non-null   int64  
 4   incident_created_hour                      4886 non-null   int64  
 5   incident_created_minute                    4886 non-null   int64  
 6   incident_latitude                          4886 non-null   float64
 7   incident_longitude                         4886 non-null   float64
 8   incident_geohash                           4886 non-null   object 
 9   LCD_station_id                             4886 non-null   int64  
 10  LCD_station_elevation   

### Checking for outliers

In [ ]:
desc_df = df1_wo_nan_rows_cols.describe()
styled_df = desc_df.style.set_sticky(axis="index")
styled_df

,incident_created_year,incident_created_month,incident_created_day,incident_created_hour,incident_created_minute,incident_latitude,incident_longitude,LCD_station_id,LCD_station_elevation,LCD_station_distance,GHCN_station_elevation,GHCN_station_distance,sumPrecipitation,dist_sumPrecipitation,maxDryBulbTemperature,dist_maxDryBulbTemperature,minDryBulbTemperature,dist_minDryBulbTemperature,meanDryBulbTemperature,dist_meanDryBulbTemperature,meanDewPointTemperature,dist_meanDewPointTemperature,meanWetBulbTemperature,dist_meanWetBulbTemperature,meanWindSpeed,dist_meanWindSpeed,meanRelativeHumidity,dist_meanRelativeHumidity,minRelativeHumidity,dist_minRelativeHumidity,maxRelativeHumidity,dist_maxRelativeHumidity,maxWindSpeed,dist_maxWindSpeed,calculate_circular_meanWindDirection,dist_calculate_circular_meanWindDirection,mode_functionWindDirection,dist_mode_functionWindDirection,dist_mode_functionSkyConditions,10_sumPrecipitation,10_maxDryBulbTemperature,10_minDryBulbTemperature,10_meanDryBulbTemperature,10_meanDewPointTemperature,10_meanWetBulbTemperature,10_meanWindSpeed,10_meanRelativeHumidity,10_minRelativeHumidity,10_maxRelativeHumidity,10_maxWindSpeed,10_calculate_circular_meanWindDirection,10_mode_functionWindDirection,30_sumPrecipitation,30_maxDryBulbTemperature,30_minDryBulbTemperature,30_meanDryBulbTemperature,30_meanDewPointTemperature,30_meanWetBulbTemperature,30_meanWindSpeed,30_meanRelativeHumidity,30_minRelativeHumidity,30_maxRelativeHumidity,30_maxWindSpeed,30_calculate_circular_meanWindDirection,30_mode_functionWindDirection,60_sumPrecipitation,60_maxDryBulbTemperature,60_minDryBulbTemperature,60_meanDryBulbTemperature,60_meanDewPointTemperature,60_meanWetBulbTemperature,60_meanWindSpeed,60_meanRelativeHumidity,60_minRelativeHumidity,60_maxRelativeHumidity,60_maxWindSpeed,60_calculate_circular_meanWindDirection,60_mode_functionWindDirection,far_hist_avg_acres_burned,near_hist_avg_acres_burned,class_label
count,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000
mean,2018.586574,6.616251,15.562833,12.426525,29.337495,37.252291,-119.956848,41265.763610,443.308473,33.035801,560.356672,16.614436,0.020970,16.880302,72.766721,16.172448,58.467433,16.172448,64.786552,33.035801,41.590584,33.035801,52.595256,33.035801,6.577201,16.178938,51.634892,33.035801,28.703438,33.035801,74.090258,33.035801,14.796152,33.035801,232.074695,33.035801,94.604994,33.035801,33.035801,0.020689,72.533937,58.251816,64.243963,41.990606,52.511378,6.594827,52.942250,29.790069,75.085377,14.445445,232.213191,41.725338,0.021151,72.174906,58.010799,63.874588,42.086045,52.383295,6.617446,53.640830,30.390011,75.760631,14.475524,233.593472,33.493655,0.020790,71.807986,57.640278,63.237640,41.902977,52.036589,6.627268,54.301379,30.966754,76.320921,14.468457,233.434209,29.250921,1574.293573,804.844076,0.325215
std,2.387655,2.987503,8.831184,6.194380,17.454585,2.573709,2.273117,36485.093228,543.723605,22.301088,580.548524,14.306874,0.167743,14.274780,9.201201,13.237641,9.191357,13.237641,15.267931,22.301088,12.448083,22.301088,10.217706,22.301088,2.588044,13.30

In [ ]:
features = [
    "sumPrecipitation", "maxDryBulbTemperature", "minDryBulbTemperature",
    "meanDryBulbTemperature", "meanDewPointTemperature", "meanWetBulbTemperature",
    "meanWindSpeed", "meanRelativeHumidity", "minRelativeHumidity",
    "maxRelativeHumidity", "maxWindSpeed", "calculate_circular_meanWindDirection",
    "mode_functionWindDirection", "mode_functionSkyConditions",
    "10_sumPrecipitation", "10_maxDryBulbTemperature", "10_minDryBulbTemperature",
    "10_meanDryBulbTemperature", "10_meanDewPointTemperature", "10_meanWetBulbTemperature",
    "10_meanWindSpeed", "10_meanRelativeHumidity", "10_minRelativeHumidity",
    "10_maxRelativeHumidity", "10_maxWindSpeed", "10_calculate_circular_meanWindDirection",
    "10_mode_functionWindDirection", "10_mode_functionSkyConditions",
    "30_sumPrecipitation", "30_maxDryBulbTemperature", "30_minDryBulbTemperature",
    "30_meanDryBulbTemperature", "30_meanDewPointTemperature", "30_meanWetBulbTemperature",
    "30_meanWindSpeed", "30_meanRelativeHumidity", "30_minRelativeHumidity",
    "30_maxRelativeHumidity", "30_maxWindSpeed", "30_calculate_circular_meanWindDirection",
    "30_mode_functionWindDirection", "30_mode_functionSkyConditions",
    "60_sumPrecipitation", "60_maxDryBulbTemperature", "60_minDryBulbTemperature",
    "60_meanDryBulbTemperature", "60_meanDewPointTemperature", "60_meanWetBulbTemperature",
    "60_meanWindSpeed", "60_meanRelativeHumidity", "60_minRelativeHumidity",
    "60_maxRelativeHumidity", "60_maxWindSpeed", "60_calculate_circular_meanWindDirection",
    "60_mode_functionWindDirection", "60_mode_functionSkyConditions",
    "far_hist_avg_acres_burned", "near_hist_avg_acres_burned"
]


acceptable_ranges = {
    "sumPrecipitation": (0, float('inf')),  # ≥ 0 inches
    "maxDryBulbTemperature": (-58, 122),  # -58 to 122 °F
    "minDryBulbTemperature": (-58, 122),
    "meanDryBulbTemperature": (-58, 122),
    "meanDewPointTemperature": (-22, 86),  # -22 to 86 °F
    "meanWetBulbTemperature": (-22, 86),
    "meanWindSpeed": (0, 200),  # 0 to 200 mph
    "meanRelativeHumidity": (0, 100),  # 0 - 100%
    "minRelativeHumidity": (0, 100),
    "maxRelativeHumidity": (0, 100),
    "maxWindSpeed": (0, 200),
    "calculate_circular_meanWindDirection": (0, 360),  # 0 - 360 degrees
    "mode_functionWindDirection": (0, 360),
    # 10_ prefix
    "10_sumPrecipitation": (0, float('inf')),  # ≥ 0 inches
    "10_maxDryBulbTemperature": (-58, 122),
    "10_minDryBulbTemperature": (-58, 122),
    "10_meanDryBulbTemperature": (-58, 122),
    "10_meanDewPointTemperature": (-22, 86),
    "10_meanWetBulbTemperature": (-22, 86),
    "10_meanWindSpeed": (0, 200),
    "10_meanRelativeHumidity": (0, 100),
    "10_minRelativeHumidity": (0, 100),
    "10_maxRelativeHumidity": (0, 100),
    "10_maxWindSpeed": (0, 200),
    "10_calculate_circular_meanWindDirection": (0, 360),
    "10_mode_functionWindDirection": (0, 360),
    # 30_ prefix
    "30_sumPrecipitation": (0, float('inf')),  # ≥ 0 inches
    "30_maxDryBulbTemperature": (-58, 122),
    "30_minDryBulbTemperature": (-58, 122),
    "30_meanDryBulbTemperature": (-58, 122),
    "30_meanDewPointTemperature": (-22, 86),
    "30_meanWetBulbTemperature": (-22, 86),
    "30_meanWindSpeed": (0, 200),
    "30_meanRelativeHumidity": (0, 100),
    "30_minRelativeHumidity": (0, 100),
    "30_maxRelativeHumidity": (0, 100),
    "30_maxWindSpeed": (0, 200),
    "30_calculate_circular_meanWindDirection": (0, 360),
    "30_mode_functionWindDirection": (0, 360),
    # 60_ prefix
    "60_sumPrecipitation": (0, float('inf')),  # ≥ 0 inches
    "60_maxDryBulbTemperature": (-58, 122),
    "60_minDryBulbTemperature": (-58, 122),
    "60_meanDryBulbTemperature": (-58, 122),
    "60_meanDewPointTemperature": (-22, 86),
    "60_meanWetBulbTemperature": (-22, 86),
    "60_meanWindSpeed": (0, 200),
    "60_meanRelativeHumidity": (0, 100),
    "60_minRelativeHumidity": (0, 100),
    "60_maxRelativeHumidity": (0, 100),
    "60_maxWindSpeed": (0, 200),
    "60_calculate_circular_meanWindDirection": (0, 360),
    "60_mode_functionWindDirection": (0, 360),
    # Additional features
    "far_hist_avg_acres_burned": (0, float('inf')),  # ≥ 0 acres
    "near_hist_avg_acres_burned": (0, float('inf')),
}


def check_range(value, range_tuple):
    return range_tuple[0] <= value <= range_tuple[1]

def print_min_max_mean_values(df, features):
    in_range_features = []
    out_of_range_features = []
    mean_out_of_bounds_features = []

    for feature in features:
        if feature in df.columns and df[feature].dtype in ['float64', 'int64']:
            min_value = df[feature].min()
            max_value = df[feature].max()
            mean_value = df[feature].mean()

            # Check if min and max values are within acceptable range
            min_check = check_range(min_value, acceptable_ranges.get(feature, (float('-inf'), float('inf'))))
            max_check = check_range(max_value, acceptable_ranges.get(feature, (float('-inf'), float('inf'))))

            # Check if mean value is within min and max of the feature
            mean_within_bounds = min_value <= mean_value <= max_value

            # Status indicators
            min_status = '✓' if min_check else '✕'
            max_status = '✓' if max_check else '✕'
            mean_status = '✓' if mean_within_bounds else '✕'

            # Improved formatting with rounded values
            feature_info = (f"{feature:<35} "
                            f"PhyMin = {acceptable_ranges.get(feature)[0]:<10} "
                            f"PhyMax = {acceptable_ranges.get(feature)[1]:<10} "
                            f"{min_status} Min = {min_value:<10.5f} "
                            f"{max_status} Max = {max_value:<10.5f} "
                            f"{mean_status} Mean = {mean_value:<10.5f}")
            if min_check and max_check:
                if mean_within_bounds:
                    in_range_features.append(feature_info)
                else:
                    mean_out_of_bounds_features.append(feature_info)
            else:
                out_of_range_features.append(feature_info)

    # Print features with in-range values
    print("Features Within Acceptable Range:")
    for feature_info in in_range_features:
        print(feature_info)

    # Print features with mean values out of min-max bounds
    print("\nFeatures with Mean Out of Min-Max Bounds:")
    for feature_info in mean_out_of_bounds_features:
        print(feature_info)

    # Print features with out-of-range min or max values
    print("\nFeatures with Out-of-Range Min or Max Values:")
    for feature_info in out_of_range_features:
        print(feature_info)

# Call the function with the dataframe and features
print_min_max_mean_values(df1_wo_nan_rows_cols, features)




Features Within Acceptable Range:
sumPrecipitation                    PhyMin = 0          PhyMax = inf        ✓ Min = 0.00000    ✓ Max = 5.49000    ✓ Mean = 0.02097   
maxDryBulbTemperature               PhyMin = -58        PhyMax = 122        ✓ Min = 32.00000   ✓ Max = 121.00000  ✓ Mean = 72.76672  
minDryBulbTemperature               PhyMin = -58        PhyMax = 122        ✓ Min = 1.00000    ✓ Max = 98.00000   ✓ Mean = 58.46743  
meanDryBulbTemperature              PhyMin = -58        PhyMax = 122        ✓ Min = 8.77358    ✓ Max = 109.15385  ✓ Mean = 64.78655  
meanDewPointTemperature             PhyMin = -22        PhyMax = 86         ✓ Min = -18.64286  ✓ Max = 76.16667   ✓ Mean = 41.59058  
meanWetBulbTemperature              PhyMin = -22        PhyMax = 86         ✓ Min = 8.15094    ✓ Max = 82.00000   ✓ Mean = 52.59526  
meanWindSpeed                       PhyMin = 0          PhyMax = 200        ✓ Min = 0.00000    ✓ Max = 32.45714   ✓ Mean = 6.57720   
meanRelativeHumidity        

###  Outliers (impossible values) removed by correcting the initial source script that converts hourly measurements to daily measurements. Double checking below

In [ ]:
# Define the lower and upper bounds for dew point temperature
lower_bound = -22  # -22°F
upper_bound = 86   # 86°F

# Filter the DataFrame for rows where any dew point temperature is out of range
out_of_range_rows = df1_wo_nan_rows_cols[
    (df1_wo_nan_rows_cols["meanDewPointTemperature"] < lower_bound) |
    (df1_wo_nan_rows_cols["meanDewPointTemperature"] > upper_bound) |
    (df1_wo_nan_rows_cols["10_meanDewPointTemperature"] < lower_bound) |
    (df1_wo_nan_rows_cols["10_meanDewPointTemperature"] > upper_bound) |
    (df1_wo_nan_rows_cols["30_meanDewPointTemperature"] < lower_bound) |
    (df1_wo_nan_rows_cols["30_meanDewPointTemperature"] > upper_bound) |
    (df1_wo_nan_rows_cols["60_meanDewPointTemperature"] < lower_bound) |
    (df1_wo_nan_rows_cols["60_meanDewPointTemperature"] > upper_bound)
]

# Display the rows
out_of_range_rows[["incident_name", "incident_latitude", "incident_longitude", "meanDewPointTemperature", "10_meanDewPointTemperature", "30_meanDewPointTemperature", "60_meanDewPointTemperature"]]


,incident_name,incident_latitude,incident_longitude,meanDewPointTemperature,10_meanDewPointTemperature,30_meanDewPointTemperature,60_meanDewPointTemperature


In [ ]:
# Define the lower and upper bounds for wind speed
lower_bound = 0   # 0 mph
upper_bound = 200 # 200 mph

# Filter the DataFrame for rows where any wind speed (mean or max) is out of range
out_of_range_rows = df1_wo_nan_rows_cols[
    (df1_wo_nan_rows_cols["meanWindSpeed"] < lower_bound) |
    (df1_wo_nan_rows_cols["meanWindSpeed"] > upper_bound) |
    (df1_wo_nan_rows_cols["10_meanWindSpeed"] < lower_bound) |
    (df1_wo_nan_rows_cols["10_meanWindSpeed"] > upper_bound) |
    (df1_wo_nan_rows_cols["30_meanWindSpeed"] < lower_bound) |
    (df1_wo_nan_rows_cols["30_meanWindSpeed"] > upper_bound) |
    (df1_wo_nan_rows_cols["60_meanWindSpeed"] < lower_bound) |
    (df1_wo_nan_rows_cols["60_meanWindSpeed"] > upper_bound) |
    (df1_wo_nan_rows_cols["maxWindSpeed"] < lower_bound) |
    (df1_wo_nan_rows_cols["maxWindSpeed"] > upper_bound) |
    (df1_wo_nan_rows_cols["10_maxWindSpeed"] < lower_bound) |
    (df1_wo_nan_rows_cols["10_maxWindSpeed"] > upper_bound) |
    (df1_wo_nan_rows_cols["30_maxWindSpeed"] < lower_bound) |
    (df1_wo_nan_rows_cols["30_maxWindSpeed"] > upper_bound) |
    (df1_wo_nan_rows_cols["60_maxWindSpeed"] < lower_bound) |
    (df1_wo_nan_rows_cols["60_maxWindSpeed"] > upper_bound)
]

# Display the rows
out_of_range_rows[["incident_name", "incident_latitude", "incident_longitude", "meanWindSpeed", "10_meanWindSpeed", "30_meanWindSpeed", "60_meanWindSpeed", "maxWindSpeed", "10_maxWindSpeed", "30_maxWindSpeed", "60_maxWindSpeed"]]


,incident_name,incident_latitude,incident_longitude,meanWindSpeed,10_meanWindSpeed,30_meanWindSpeed,60_meanWindSpeed,maxWindSpeed,10_maxWindSpeed,30_maxWindSpeed,60_maxWindSpeed


### Duplicate check

In [ ]:
# Find duplicates
duplicate_rows = df1_wo_nan_rows_cols.duplicated()

num_duplicates = duplicate_rows.sum()
print(f"Number of duplicate rows: {num_duplicates}")

if num_duplicates > 0:
    print("Duplicate Rows:")
    print(df1_wo_nan_rows_cols[duplicate_rows])


Number of duplicate rows: 0


### Viewing the finished dataframe

In [ ]:
df1_wo_nan_rows_cols

,incident_name,incident_created_year,incident_created_month,incident_created_day,incident_created_hour,incident_created_minute,incident_latitude,incident_longitude,incident_geohash,LCD_station_id,LCD_station_elevation,LCD_station_distance,GHCN_station_id,GHCN_station_elevation,GHCN_station_distance,sumPrecipitation,dist_sumPrecipitation,maxDryBulbTemperature,dist_maxDryBulbTemperature,minDryBulbTemperature,dist_minDryBulbTemperature,meanDryBulbTemperature,dist_meanDryBulbTemperature,meanDewPointTemperature,dist_meanDewPointTemperature,meanWetBulbTemperature,dist_meanWetBulbTemperature,meanWindSpeed,dist_meanWindSpeed,meanRelativeHumidity,dist_meanRelativeHumidity,minRelativeHumidity,dist_minRelativeHumidity,maxRelativeHumidity,dist_maxRelativeHumidity,maxWindSpeed,dist_maxWindSpeed,calculate_circular_meanWindDirection,dist_calculate_circular_meanWindDirection,mode_functionWindDirection,dist_mode_functionWindDirection,mode_functionSkyConditions,dist_mode_functionSkyConditions,10_sumPrecipitation,10_maxDryBulbTemperature,10_minDryBulbTemperature,10_meanDryBulbTemperature,10_meanDewPointTemperature,10_meanWetBulbTemperature,10_meanWindSpeed,10_meanRelativeHumidity,10_minRelativeHumidity,10_maxRelativeHumidity,10_maxWindSpeed,10_calculate_circular_meanWindDirection,10_mode_functionWindDirection,10_mode_functionSkyConditions,30_sumPrecipitation,30_maxDryBulbTemperature,30_minDryBulbTemperature,30_meanDryBulbTemperature,30_meanDewPointTemperature,30_meanWetBulbTemperature,30_meanWindSpeed,30_meanRelativeHumidity,30_minRelativeHumidity,30_maxRelativeHumidity,30_maxWindSpeed,30_calculate_circular_meanWindDirection,30_mode_functionWindDirection,30_mode_functionSkyConditions,60_sumPrecipitation,60_maxDryBulbTemperature,60_minDryBulbTemperature,60_meanDryBulbTemperature,60_meanDewPointTemperature,60_meanWetBulbTemperature,60_meanWindSpeed,60_meanRelativeHumidity,60_minRelativeHumidity,60_maxRelativeHumidity,60_maxWindSpeed,60_calculate_circular_meanWindDirection,60_mode_functionWindDirection,60_mode_functionSkyConditions,far_hist_avg_acres_burned,near_hist_avg_acres_burned,class_label
0,Stephens Fire,2015,2,24,12,15,41.485000,-121.851000,9r3k50n,24215,1083.0,43.235962,USC00045449,985.4,35.323794,0.0,35.323794,66.02,35.323794,48.02,35.323794,43.375000,43.235962,8.833333,43.235962,31.166667,43.235962,6.039900,35.323794,27.458333,43.235962,11.0,43.235962,51.0,43.235962,11.0,43.235962,5.809858,43.235962,0.0,43.235962,CLR:00,43.235962,0.011024,65.876,55.508,44.041667,23.608333,35.737500,5.659610,49.112500,24.7,72.1,10.1,11.502194,0.0,CLR:00,0.004856,69.494000,53.762000,44.375856,31.921307,39.020870,5.450823,66.276835,44.366667,83.533333,8.333333,19.407850,0.0,CLR:00,0.007283,67.958000,50.594000,41.523426,30.378900,36.800077,4.947498,68.275119,45.700000,84.450000,8.000000,12.517703,0.0,CLR:00,39.6260,136.333333,1
1,Ward Fire,2015,4,13,5,30,40.050833,-120.701667,9r45v75,23225,1611.2,86.141091,USC00047195,1042.4,24.487379,0.0,24.487379,69.08,24.487379,57.92,24.487379,50.153846,86.141091,27.500000,86.141091,39.923077,86.141091,6.039900,24.487379,43.000000,86.141091,27.0,86.141091,79.0,86.141091,18.0,86.141091,163.864889,86.141091,170.0,86.141091,CLR:00,86.141091,0.000000,68.090,55.490,40.773436,22.318814,33.373392,7.337360,54.611732,36.6,69.8,12.3,138.314704,0.0,CLR:00,0.000000,71.336000,57.302000,47.705004,25.581872,38.145478,6.278513,47.408972,31.700000,64.033333,12.800000,125.826118,0.0,CLR:00,0.017979,70.166000,55.202000,48.141038,23.987898,38.000429,6.189033,44.454664,27.850000,62.950000,11.666667,73.118840,0.0,CLR:00,1444.3735,46.090000,1
2,Highway Fire,2015,4,18,18,12,33.884313,-117.642759,9qh2fbr,53175,162.5,4.007019,USW00053175,162.5,4.007226,0.0,4.007019,83.00,4.007019,38.00,4.007019,63.500000,4.007019,37.916667,4.007019,50.458333,4.007019,4.083333,4.007019,45.166667,4.007019,19.0,4.007019,80.0,4.007019,15.0,4.007019,328.751345,4.007019,0.0,4.007019,CLR:00,4.007019,0.000000,79.100,42.100,61.461923,37.971859,49.296875,4.848910,52.553141,

In [ ]:
df1_wo_nan_rows_cols.groupby("class_label").describe()

incident_created_year                                         \
                            count         mean       std     min     25%   
class_label                                                                
0                          3297.0  2018.552927  2.504427  2015.0  2016.0   
1                          1589.0  2018.656388  2.124048  2015.0  2017.0   

                                    incident_created_month            \
                50%     75%     max                  count      mean   
class_label                                                            
0            2018.0  2021.0  2023.0                 3297.0  6.316045   
1            2018.0  2020.0  2023.0                 1589.0  7.239144   

                                                incident_created_day  \
                  std  min  25%  50%  75%   max                count   
class_label                                                            
0            3.388421  1.0  3.0  6.0  9.0  12.0               3297.0   
1            1.746375  1.0  6.0  7.0  8.0  12.0               1589.0   

                                                              \
                  mean       std  min  25%   50%   75%   max   
class_label                                                    
0            15.577191  8.790244  1.0  8.0  15.0  23.0  31.0   
1            15.533040  8.918237  1.0  8.0  15.0  23.0  31.0   

            incident_created_hour                                              \
                            count       mean       std  min   25%   50%   75%   
class_label                                                                     
0                          3297.0  11.571125  6.885805  0.0   6.0  12.0  17.0   
1                          1589.0  14.201385  3.866764  0.0  12.0  14.0  16.0   

                  incident_created_minute                                   \
              max                   count       mean        std  min   25%   
class_label                                                                  
0            23.0                  3297.0  29.574158  17.437789  0.0  14.0   
1            23.0                  1589.0  28.846444  17.484656  0.0  14.0   

                              incident_latitude                       \
              50%   75%   max             count       mean       std   
class_label                                                            
0            30.0  45.0  59.0            3297.0  37.255051  2.586954   
1            30.0  44.0  59.0            1589.0  37.246565  2.546808   

                                                                   \
                   min       25%        50%        75%        max   
class_label                                                         
0            32.569660  34.99972  37.113010  39.460240  42.006310   
1            32.557546  35.05747  37.433192  39.319145  41.999341   

            incident_longitude                                              \
                         count        mean       std        min        25%   
class_label                                                                  
0                       3297.0 -119.798436  2.377364 -124.34718 -121.60332   
1                       1589.0 -120.285534  2.000968 -124.19954 -121.85300   

                                              LCD_station_id                \
                   50%        75%         max          count          mean   
class_label                                                                  
0           -120.29016 -118.00376 -114.286910         3297.0  40253.443433   
1           -120.71907 -118.80123 -114.484173         1589.0  43366.216488   

                                                                      \
                      std    min      25%      50%      75%      max   
class_label                                                            
0            36308.976812  115.0  23119.0  23257.0  93138.0  94299.0   
1            3

## Part 2 - Second data set with nan rows imputed (5817 count)
## DF2 = Null columns dropped, null rows maintained, no impossible outliers. Nans imputed with mean, median and mode.
## DF3 = Null columns dropped, null rows maintained, no impossible outliers. Nans imputed with mean (no median) and mode.

### Read in data

In [ ]:
df2 = pd.read_csv(DATASET_FILENAME)
df3 = pd.read_csv(DATASET_FILENAME)

In [ ]:
df2.shape

(5817, 133)

### Drop all/mostly null columns

In [ ]:
all_mostly_null_cols = ["SNOW", "dist_SNOW", "SNWD", "dist_SNWD", "EVAP", "dist_EVAP",
                 "FMTM", "dist_FMTM", "FRGB", "dist_FRGB", "FRGT", "dist_FRGT",
                 "FRTH", "dist_FRTH", "maxWindGustSpeed", "dist_maxWindGustSpeed",
                 "mode_functionPresentWeatherType",
                 "dist_mode_functionPresentWeatherType", "10_SNOW", "10_SNWD",
                 "10_EVAP", "10_FMTM", "10_FRGB", "10_FRGT", "10_FRTH",
                 "10_maxWindGustSpeed", "10_mode_functionPresentWeatherType",
                 "30_SNOW", "30_SNWD", "30_EVAP", "30_FMTM", "30_FRGB",
                 "30_FRGT", "30_FRTH", "30_maxWindGustSpeed",
                 "30_mode_functionPresentWeatherType", "60_SNOW", "60_SNWD",
                 "60_EVAP", "60_FMTM", "60_FRGB", "60_FRGT", "60_FRTH",
                  "60_maxWindGustSpeed", "60_mode_functionPresentWeatherType", ]
df2_wo_mostly_null_cols = df2.drop(all_mostly_null_cols, axis=1)
df3_wo_mostly_null_cols = df3.drop(all_mostly_null_cols, axis=1)

#### View Updated info without null columns

In [ ]:
print(df2_wo_mostly_null_cols.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5817 entries, 0 to 5816
Data columns (total 88 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   incident_name                              5817 non-null   object 
 1   incident_created_year                      5817 non-null   int64  
 2   incident_created_month                     5817 non-null   int64  
 3   incident_created_day                       5817 non-null   int64  
 4   incident_created_hour                      5817 non-null   int64  
 5   incident_created_minute                    5817 non-null   int64  
 6   incident_latitude                          5817 non-null   float64
 7   incident_longitude                         5817 non-null   float64
 8   incident_geohash                           5817 non-null   object 
 9   LCD_station_id                             5817 non-null   int64  
 10  LCD_station_elevation   

In [ ]:
df2_wo_mostly_null_cols.shape

(5817, 88)

### Checking for outliers

In [ ]:
desc_df = df2_wo_mostly_null_cols.describe()
styled_df = desc_df.style.set_sticky(axis="index")
styled_df

,incident_created_year,incident_created_month,incident_created_day,incident_created_hour,incident_created_minute,incident_latitude,incident_longitude,LCD_station_id,LCD_station_elevation,LCD_station_distance,GHCN_station_elevation,GHCN_station_distance,sumPrecipitation,dist_sumPrecipitation,maxDryBulbTemperature,dist_maxDryBulbTemperature,minDryBulbTemperature,dist_minDryBulbTemperature,meanDryBulbTemperature,dist_meanDryBulbTemperature,meanDewPointTemperature,dist_meanDewPointTemperature,meanWetBulbTemperature,dist_meanWetBulbTemperature,meanWindSpeed,dist_meanWindSpeed,meanRelativeHumidity,dist_meanRelativeHumidity,minRelativeHumidity,dist_minRelativeHumidity,maxRelativeHumidity,dist_maxRelativeHumidity,maxWindSpeed,dist_maxWindSpeed,calculate_circular_meanWindDirection,dist_calculate_circular_meanWindDirection,mode_functionWindDirection,dist_mode_functionWindDirection,dist_mode_functionSkyConditions,10_sumPrecipitation,10_maxDryBulbTemperature,10_minDryBulbTemperature,10_meanDryBulbTemperature,10_meanDewPointTemperature,10_meanWetBulbTemperature,10_meanWindSpeed,10_meanRelativeHumidity,10_minRelativeHumidity,10_maxRelativeHumidity,10_maxWindSpeed,10_calculate_circular_meanWindDirection,10_mode_functionWindDirection,30_sumPrecipitation,30_maxDryBulbTemperature,30_minDryBulbTemperature,30_meanDryBulbTemperature,30_meanDewPointTemperature,30_meanWetBulbTemperature,30_meanWindSpeed,30_meanRelativeHumidity,30_minRelativeHumidity,30_maxRelativeHumidity,30_maxWindSpeed,30_calculate_circular_meanWindDirection,30_mode_functionWindDirection,60_sumPrecipitation,60_maxDryBulbTemperature,60_minDryBulbTemperature,60_meanDryBulbTemperature,60_meanDewPointTemperature,60_meanWetBulbTemperature,60_meanWindSpeed,60_meanRelativeHumidity,60_minRelativeHumidity,60_maxRelativeHumidity,60_maxWindSpeed,60_calculate_circular_meanWindDirection,60_mode_functionWindDirection,far_hist_avg_acres_burned,near_hist_avg_acres_burned,class_label
count,5817.000000,5817.000000,5817.000000,5817.000000,5817.000000,5817.000000,5817.000000,5817.000000,5817.000000,5817.000000,5817.000000,5817.000000,5810.000000,5810.000000,5813.000000,5813.000000,5813.000000,5813.000000,5575.000000,5575.000000,5233.000000,5233.000000,5203.000000,5203.000000,5814.000000,5814.000000,5234.000000,5234.000000,5234.000000,5234.000000,5234.000000,5234.000000,5584.000000,5584.000000,5241.000000,5241.000000,5241.000000,5241.000000,5135.000000,5793.000000,5810.000000,5810.000000,5570.000000,5228.000000,5192.000000,5809.000000,5229.000000,5229.000000,5229.000000,5581.000000,5238.000000,5238.000000,5790.000000,5812.000000,5812.000000,5571.000000,5228.000000,5195.000000,5813.000000,5229.000000,5229.000000,5229.000000,5580.000000,5237.000000,5237.000000,5768.000000,5811.000000,5811.000000,5570.000000,5227.000000,5191.000000,5813.000000,5228.000000,5228.000000,5228.000000,5576.000000,5233.000000,5233.000000,5705.000000,5749.000000,5817.000000
mean,2018.654633,6.658931,15.606670,12.439574,29.229156,37.241487,-119.943875,41173.706206,445.810865,32.696262,547.096906,16.845154,0.019784,17.029613,72.863575,16.437329,58.567772,16.436069,64.954948,32.520356,41.664285,32.778709,52.703905,32.826697,6.540167,16.442372,51.504176,32.785798,28.734047,32.785798,73.951471,32.785798,14.362643,32.496074,231.652430,32.758620,95.002862,32.758620,32.926783,0.019324,72.650752,58.380828,64.410492,42.088587,52.630662,6.558398,52.844224,29.806027,75.006082,14.006534,232.131424,40.864834,0.020196,72.284710,58.156150,64.031751,42.184835,52.489821,6.581814,53.556310,30.430528,75.692150,14.026517,233.777591,33.064732,0.019812,71.898449,57.779098,63.375967,41.993258,52.132679,6.597151,54.220576,31.021096,76.255017,14.031553,233.152429,28.876362,1671.266546,874.153566,0.333333
std,2.384655,2.987115,8.884074,6.189647,17.545635,2.567358,2.285407,35660.397537,556.490223,21.810143,573.574634,14.454108,0.160572,14.355311,9.212596,13.516726,9.132374,13.517441,15.219894,21.855450,12.476098,22.165340,10.194425,22.178825,2.540013,13.57

In [ ]:
features = [
    "sumPrecipitation", "maxDryBulbTemperature", "minDryBulbTemperature",
    "meanDryBulbTemperature", "meanDewPointTemperature", "meanWetBulbTemperature",
    "meanWindSpeed", "meanRelativeHumidity", "minRelativeHumidity",
    "maxRelativeHumidity", "maxWindSpeed", "calculate_circular_meanWindDirection",
    "mode_functionWindDirection", "mode_functionSkyConditions",
    "10_sumPrecipitation", "10_maxDryBulbTemperature", "10_minDryBulbTemperature",
    "10_meanDryBulbTemperature", "10_meanDewPointTemperature", "10_meanWetBulbTemperature",
    "10_meanWindSpeed", "10_meanRelativeHumidity", "10_minRelativeHumidity",
    "10_maxRelativeHumidity", "10_maxWindSpeed", "10_calculate_circular_meanWindDirection",
    "10_mode_functionWindDirection", "10_mode_functionSkyConditions",
    "30_sumPrecipitation", "30_maxDryBulbTemperature", "30_minDryBulbTemperature",
    "30_meanDryBulbTemperature", "30_meanDewPointTemperature", "30_meanWetBulbTemperature",
    "30_meanWindSpeed", "30_meanRelativeHumidity", "30_minRelativeHumidity",
    "30_maxRelativeHumidity", "30_maxWindSpeed", "30_calculate_circular_meanWindDirection",
    "30_mode_functionWindDirection", "30_mode_functionSkyConditions",
    "60_sumPrecipitation", "60_maxDryBulbTemperature", "60_minDryBulbTemperature",
    "60_meanDryBulbTemperature", "60_meanDewPointTemperature", "60_meanWetBulbTemperature",
    "60_meanWindSpeed", "60_meanRelativeHumidity", "60_minRelativeHumidity",
    "60_maxRelativeHumidity", "60_maxWindSpeed", "60_calculate_circular_meanWindDirection",
    "60_mode_functionWindDirection", "60_mode_functionSkyConditions",
    "far_hist_avg_acres_burned", "near_hist_avg_acres_burned"
]


acceptable_ranges = {
    "sumPrecipitation": (0, float('inf')),  # ≥ 0 inches
    "maxDryBulbTemperature": (-58, 122),  # -58 to 122 °F
    "minDryBulbTemperature": (-58, 122),
    "meanDryBulbTemperature": (-58, 122),
    "meanDewPointTemperature": (-22, 86),  # -22 to 86 °F
    "meanWetBulbTemperature": (-22, 86),
    "meanWindSpeed": (0, 200),  # 0 to 200 mph
    "meanRelativeHumidity": (0, 100),  # 0 - 100%
    "minRelativeHumidity": (0, 100),
    "maxRelativeHumidity": (0, 100),
    "maxWindSpeed": (0, 200),
    "calculate_circular_meanWindDirection": (0, 360),  # 0 - 360 degrees
    "mode_functionWindDirection": (0, 360),
    # 10_ prefix
    "10_sumPrecipitation": (0, float('inf')),  # ≥ 0 inches
    "10_maxDryBulbTemperature": (-58, 122),
    "10_minDryBulbTemperature": (-58, 122),
    "10_meanDryBulbTemperature": (-58, 122),
    "10_meanDewPointTemperature": (-22, 86),
    "10_meanWetBulbTemperature": (-22, 86),
    "10_meanWindSpeed": (0, 200),
    "10_meanRelativeHumidity": (0, 100),
    "10_minRelativeHumidity": (0, 100),
    "10_maxRelativeHumidity": (0, 100),
    "10_maxWindSpeed": (0, 200),
    "10_calculate_circular_meanWindDirection": (0, 360),
    "10_mode_functionWindDirection": (0, 360),
    # 30_ prefix
    "30_sumPrecipitation": (0, float('inf')),  # ≥ 0 inches
    "30_maxDryBulbTemperature": (-58, 122),
    "30_minDryBulbTemperature": (-58, 122),
    "30_meanDryBulbTemperature": (-58, 122),
    "30_meanDewPointTemperature": (-22, 86),
    "30_meanWetBulbTemperature": (-22, 86),
    "30_meanWindSpeed": (0, 200),
    "30_meanRelativeHumidity": (0, 100),
    "30_minRelativeHumidity": (0, 100),
    "30_maxRelativeHumidity": (0, 100),
    "30_maxWindSpeed": (0, 200),
    "30_calculate_circular_meanWindDirection": (0, 360),
    "30_mode_functionWindDirection": (0, 360),
    # 60_ prefix
    "60_sumPrecipitation": (0, float('inf')),  # ≥ 0 inches
    "60_maxDryBulbTemperature": (-58, 122),
    "60_minDryBulbTemperature": (-58, 122),
    "60_meanDryBulbTemperature": (-58, 122),
    "60_meanDewPointTemperature": (-22, 86),
    "60_meanWetBulbTemperature": (-22, 86),
    "60_meanWindSpeed": (0, 200),
    "60_meanRelativeHumidity": (0, 100),
    "60_minRelativeHumidity": (0, 100),
    "60_maxRelativeHumidity": (0, 100),
    "60_maxWindSpeed": (0, 200),
    "60_calculate_circular_meanWindDirection": (0, 360),
    "60_mode_functionWindDirection": (0, 360),
    # Additional features
    "far_hist_avg_acres_burned": (0, float('inf')),  # ≥ 0 acres
    "near_hist_avg_acres_burned": (0, float('inf')),
}


def check_range(value, range_tuple):
    return range_tuple[0] <= value <= range_tuple[1]

def print_min_max_mean_values(df, features):
    in_range_features = []
    out_of_range_features = []
    mean_out_of_bounds_features = []

    for feature in features:
        if feature in df.columns and df[feature].dtype in ['float64', 'int64']:
            min_value = df[feature].min()
            max_value = df[feature].max()
            mean_value = df[feature].mean()

            # Check if min and max values are within acceptable range
            min_check = check_range(min_value, acceptable_ranges.get(feature, (float('-inf'), float('inf'))))
            max_check = check_range(max_value, acceptable_ranges.get(feature, (float('-inf'), float('inf'))))

            # Check if mean value is within min and max of the feature
            mean_within_bounds = min_value <= mean_value <= max_value

            # Status indicators
            min_status = '✓' if min_check else '✕'
            max_status = '✓' if max_check else '✕'
            mean_status = '✓' if mean_within_bounds else '✕'

            # Improved formatting with rounded values
            feature_info = (f"{feature:<35} "
                            f"PhyMin = {acceptable_ranges.get(feature)[0]:<10} "
                            f"PhyMax = {acceptable_ranges.get(feature)[1]:<10} "
                            f"{min_status} Min = {min_value:<10.5f} "
                            f"{max_status} Max = {max_value:<10.5f} "
                            f"{mean_status} Mean = {mean_value:<10.5f}")
            if min_check and max_check:
                if mean_within_bounds:
                    in_range_features.append(feature_info)
                else:
                    mean_out_of_bounds_features.append(feature_info)
            else:
                out_of_range_features.append(feature_info)

    # Print features with in-range values
    print("Features Within Acceptable Range:")
    for feature_info in in_range_features:
        print(feature_info)

    # Print features with mean values out of min-max bounds
    print("\nFeatures with Mean Out of Min-Max Bounds:")
    for feature_info in mean_out_of_bounds_features:
        print(feature_info)

    # Print features with out-of-range min or max values
    print("\nFeatures with Out-of-Range Min or Max Values:")
    for feature_info in out_of_range_features:
        print(feature_info)

# Call the function with the dataframe and features
print_min_max_mean_values(df2_wo_mostly_null_cols, features)


Features Within Acceptable Range:
sumPrecipitation                    PhyMin = 0          PhyMax = inf        ✓ Min = 0.00000    ✓ Max = 5.49000    ✓ Mean = 0.01978   
maxDryBulbTemperature               PhyMin = -58        PhyMax = 122        ✓ Min = 27.00000   ✓ Max = 121.00000  ✓ Mean = 72.86357  
minDryBulbTemperature               PhyMin = -58        PhyMax = 122        ✓ Min = 1.00000    ✓ Max = 98.00000   ✓ Mean = 58.56777  
meanDryBulbTemperature              PhyMin = -58        PhyMax = 122        ✓ Min = 8.77358    ✓ Max = 109.22917  ✓ Mean = 64.95495  
meanDewPointTemperature             PhyMin = -22        PhyMax = 86         ✓ Min = -18.64286  ✓ Max = 76.16667   ✓ Mean = 41.66428  
meanWetBulbTemperature              PhyMin = -22        PhyMax = 86         ✓ Min = 8.15094    ✓ Max = 82.00000   ✓ Mean = 52.70391  
meanWindSpeed                       PhyMin = 0          PhyMax = 200        ✓ Min = 0.00000    ✓ Max = 32.45714   ✓ Mean = 6.54017   
meanRelativeHumidity        

###  Outliers (impossible values) removed by correcting the initial source script that converts hourly measurements to daily measurements. Double checking below

In [ ]:
# Define the lower and upper bounds for dew point temperature
lower_bound = -22  # -22°F
upper_bound = 86   # 86°F

# Filter the DataFrame for rows where any dew point temperature is out of range
out_of_range_rows = df2_wo_mostly_null_cols[
    (df2_wo_mostly_null_cols["meanDewPointTemperature"] < lower_bound) |
    (df2_wo_mostly_null_cols["meanDewPointTemperature"] > upper_bound) |
    (df2_wo_mostly_null_cols["10_meanDewPointTemperature"] < lower_bound) |
    (df2_wo_mostly_null_cols["10_meanDewPointTemperature"] > upper_bound) |
    (df2_wo_mostly_null_cols["30_meanDewPointTemperature"] < lower_bound) |
    (df2_wo_mostly_null_cols["30_meanDewPointTemperature"] > upper_bound) |
    (df2_wo_mostly_null_cols["60_meanDewPointTemperature"] < lower_bound) |
    (df2_wo_mostly_null_cols["60_meanDewPointTemperature"] > upper_bound)
]

# Display the rows
out_of_range_rows[["incident_name", "incident_latitude", "incident_longitude", "meanDewPointTemperature", "10_meanDewPointTemperature", "30_meanDewPointTemperature", "60_meanDewPointTemperature"]]


,incident_name,incident_latitude,incident_longitude,meanDewPointTemperature,10_meanDewPointTemperature,30_meanDewPointTemperature,60_meanDewPointTemperature


In [ ]:
# Define the lower and upper bounds for wind speed
lower_bound = 0   # 0 mph
upper_bound = 200 # 200 mph

# Filter the DataFrame for rows where any wind speed (mean or max) is out of range
out_of_range_rows = df2_wo_mostly_null_cols[
    (df2_wo_mostly_null_cols["meanWindSpeed"] < lower_bound) |
    (df2_wo_mostly_null_cols["meanWindSpeed"] > upper_bound) |
    (df2_wo_mostly_null_cols["10_meanWindSpeed"] < lower_bound) |
    (df2_wo_mostly_null_cols["10_meanWindSpeed"] > upper_bound) |
    (df2_wo_mostly_null_cols["30_meanWindSpeed"] < lower_bound) |
    (df2_wo_mostly_null_cols["30_meanWindSpeed"] > upper_bound) |
    (df2_wo_mostly_null_cols["60_meanWindSpeed"] < lower_bound) |
    (df2_wo_mostly_null_cols["60_meanWindSpeed"] > upper_bound) |
    (df2_wo_mostly_null_cols["maxWindSpeed"] < lower_bound) |
    (df2_wo_mostly_null_cols["maxWindSpeed"] > upper_bound) |
    (df2_wo_mostly_null_cols["10_maxWindSpeed"] < lower_bound) |
    (df2_wo_mostly_null_cols["10_maxWindSpeed"] > upper_bound) |
    (df2_wo_mostly_null_cols["30_maxWindSpeed"] < lower_bound) |
    (df2_wo_mostly_null_cols["30_maxWindSpeed"] > upper_bound) |
    (df2_wo_mostly_null_cols["60_maxWindSpeed"] < lower_bound) |
    (df2_wo_mostly_null_cols["60_maxWindSpeed"] > upper_bound)
]

# Display the rows
out_of_range_rows[["incident_name", "incident_latitude", "incident_longitude", "meanWindSpeed", "10_meanWindSpeed", "30_meanWindSpeed", "60_meanWindSpeed", "maxWindSpeed", "10_maxWindSpeed", "30_maxWindSpeed", "60_maxWindSpeed"]]


,incident_name,incident_latitude,incident_longitude,meanWindSpeed,10_meanWindSpeed,30_meanWindSpeed,60_meanWindSpeed,maxWindSpeed,10_maxWindSpeed,30_maxWindSpeed,60_maxWindSpeed



### Looking into the breakdown of NaN rows

In [ ]:
nan_counts = df2_wo_mostly_null_cols.isnull().sum(axis=1).value_counts().sort_index()

print("Counts of Rows with Different Numbers of NaN Values:")
for nan_count, rows in nan_counts.items():
    print(f"{nan_count} NaNs: {rows} rows")


Counts of Rows with Different Numbers of NaN Values:
0 NaNs: 4886 rows
1 NaNs: 86 rows
2 NaNs: 81 rows
3 NaNs: 27 rows
4 NaNs: 2 rows
5 NaNs: 103 rows
6 NaNs: 4 rows
7 NaNs: 2 rows
10 NaNs: 20 rows
13 NaNs: 3 rows
15 NaNs: 14 rows
16 NaNs: 2 rows
17 NaNs: 1 rows
20 NaNs: 2 rows
24 NaNs: 1 rows
25 NaNs: 1 rows
30 NaNs: 5 rows
31 NaNs: 2 rows
33 NaNs: 3 rows
35 NaNs: 11 rows
37 NaNs: 1 rows
40 NaNs: 332 rows
41 NaNs: 4 rows
42 NaNs: 3 rows
43 NaNs: 1 rows
45 NaNs: 1 rows
50 NaNs: 207 rows
51 NaNs: 2 rows
52 NaNs: 2 rows
53 NaNs: 1 rows
55 NaNs: 4 rows
70 NaNs: 3 rows


In [ ]:
pd.set_option('display.max_rows', None)
nan_counts_per_feature = df2_wo_mostly_null_cols.isnull().sum()
print("Counts of NaN Values per Feature:")
print(nan_counts_per_feature)
total_columns = len(df2_wo_mostly_null_cols.columns)
print("\nTotal Number of Columns:", total_columns)
pd.reset_option('display.max_rows')


Counts of NaN Values per Feature:
incident_name                                  0
incident_created_year                          0
incident_created_month                         0
incident_created_day                           0
incident_created_hour                          0
incident_created_minute                        0
incident_latitude                              0
incident_longitude                             0
incident_geohash                               0
LCD_station_id                                 0
LCD_station_elevation                          0
LCD_station_distance                           0
GHCN_station_id                                0
GHCN_station_elevation                         0
GHCN_station_distance                          0
sumPrecipitation                               7
dist_sumPrecipitation                          7
maxDryBulbTemperature                          4
dist_maxDryBulbTemperature                     4
minDryBulbTemperature              

### Identifying Numerical or Categorical Columns

In [ ]:
# Identifying numerical and categorical columns
numerical_cols = df2_wo_mostly_null_cols.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = df2_wo_mostly_null_cols.select_dtypes(include=['object']).columns

print("Numerical Columns:", numerical_cols)
print("\nCategorical Columns:", categorical_cols)


Numerical Columns: Index(['incident_created_year', 'incident_created_month',
       'incident_created_day', 'incident_created_hour',
       'incident_created_minute', 'incident_latitude', 'incident_longitude',
       'LCD_station_id', 'LCD_station_elevation', 'LCD_station_distance',
       'GHCN_station_elevation', 'GHCN_station_distance', 'sumPrecipitation',
       'dist_sumPrecipitation', 'maxDryBulbTemperature',
       'dist_maxDryBulbTemperature', 'minDryBulbTemperature',
       'dist_minDryBulbTemperature', 'meanDryBulbTemperature',
       'dist_meanDryBulbTemperature', 'meanDewPointTemperature',
       'dist_meanDewPointTemperature', 'meanWetBulbTemperature',
       'dist_meanWetBulbTemperature', 'meanWindSpeed', 'dist_meanWindSpeed',
       'meanRelativeHumidity', 'dist_meanRelativeHumidity',
       'minRelativeHumidity', 'dist_minRelativeHumidity',
       'maxRelativeHumidity', 'dist_maxRelativeHumidity', 'maxWindSpeed',
       'dist_maxWindSpeed', 'calculate_circular_meanWindD

### Skewness of the data

In [ ]:
# Lists to store column names and their skewness
skewed_cols = []
non_skewed_cols = []

# Checking the distribution of numerical columns and categorizing them
for col in numerical_cols:
    skewness = df2_wo_mostly_null_cols[col].skew()
    if -0.5 <= skewness <= 0.5:
        non_skewed_cols.append((col, skewness))
    else:
        skewed_cols.append((col, skewness))

# Sorting the columns by skewness
skewed_cols.sort(key=lambda x: x[1])
non_skewed_cols.sort(key=lambda x: x[1])

# Printing the results
print("Non-Skewed Columns:")
for col, skew in non_skewed_cols:
    print(f"{col}: {skew}")

print("\nSkewed Columns:")
for col, skew in skewed_cols:
    print(f"{col}: {skew}")


Non-Skewed Columns:
incident_created_hour: -0.3018532321717122
10_meanDryBulbTemperature: -0.14851645328593827
meanDryBulbTemperature: -0.1468814472109969
30_meanDryBulbTemperature: -0.1385751960263049
incident_created_month: -0.11984403977104877
60_meanDryBulbTemperature: -0.08179035019079113
incident_created_minute: -0.009093442402327439
60_meanRelativeHumidity: -0.008823809373860781
incident_created_day: 0.03533182203554723
30_meanRelativeHumidity: 0.05550217813012444
incident_latitude: 0.06755439170958218
10_meanRelativeHumidity: 0.09726387023501182
incident_created_year: 0.15740249065131515
meanRelativeHumidity: 0.19217175780824275
60_maxWindSpeed: 0.27500741128102923
30_maxWindSpeed: 0.2838774084735153
incident_longitude: 0.379860863193337
10_maxWindSpeed: 0.4140259044103396

Skewed Columns:
30_calculate_circular_meanWindDirection: -0.8507023766232419
60_calculate_circular_meanWindDirection: -0.8426737044408612
10_calculate_circular_meanWindDirection: -0.8206018064422135
60_maxRe

### Imputing the data

#### Using median and mean

In [ ]:
# Using mean or median depending on the skewness of the numerical data
# Using mode for the categorical data
# Calculating the values for each class independently i.e. if we use mean, median or mode we calculate it for non-fire nans using non-fire aggregation total, for fire nans we use fire aggergation totals.

numerical_cols = [
    'sumPrecipitation', 'dist_sumPrecipitation', 'maxDryBulbTemperature', 'dist_maxDryBulbTemperature',
    'minDryBulbTemperature', 'dist_minDryBulbTemperature', 'meanDryBulbTemperature', 'dist_meanDryBulbTemperature',
    'meanDewPointTemperature', 'dist_meanDewPointTemperature', 'meanWetBulbTemperature', 'dist_meanWetBulbTemperature',
    'meanWindSpeed', 'dist_meanWindSpeed', 'meanRelativeHumidity', 'dist_meanRelativeHumidity',
    'minRelativeHumidity', 'dist_minRelativeHumidity', 'maxRelativeHumidity', 'dist_maxRelativeHumidity',
    'maxWindSpeed', 'dist_maxWindSpeed', 'calculate_circular_meanWindDirection', 'dist_calculate_circular_meanWindDirection',
    'mode_functionWindDirection', 'dist_mode_functionWindDirection', 'dist_mode_functionSkyConditions', '10_sumPrecipitation', '10_maxDryBulbTemperature',
    '10_minDryBulbTemperature', '10_meanDryBulbTemperature', '10_meanDewPointTemperature', '10_meanWetBulbTemperature',
    '10_meanWindSpeed', '10_meanRelativeHumidity', '10_minRelativeHumidity', '10_maxRelativeHumidity',
    '10_maxWindSpeed', '10_calculate_circular_meanWindDirection', '10_mode_functionWindDirection',
    '30_sumPrecipitation', '30_maxDryBulbTemperature', '30_minDryBulbTemperature', '30_meanDryBulbTemperature',
    '30_meanDewPointTemperature', '30_meanWetBulbTemperature', '30_meanWindSpeed', '30_meanRelativeHumidity',
    '30_minRelativeHumidity', '30_maxRelativeHumidity', '30_maxWindSpeed', '30_calculate_circular_meanWindDirection',
    '30_mode_functionWindDirection', '60_sumPrecipitation', '60_maxDryBulbTemperature', '60_minDryBulbTemperature',
    '60_meanDryBulbTemperature', '60_meanDewPointTemperature', '60_meanWetBulbTemperature', '60_meanWindSpeed',
    '60_meanRelativeHumidity', '60_minRelativeHumidity', '60_maxRelativeHumidity', '60_maxWindSpeed',
    '60_calculate_circular_meanWindDirection', '60_mode_functionWindDirection', 'far_hist_avg_acres_burned', 'near_hist_avg_acres_burned'
]

categorical_cols_with_nans = [
    'mode_functionSkyConditions', '10_mode_functionSkyConditions', '30_mode_functionSkyConditions', '60_mode_functionSkyConditions'
]

# Iterate over each numerical column
for col in numerical_cols:
    skewness = df2_wo_mostly_null_cols[col].skew()

    # Calculate class-specific mean or median
    mean_or_median_non_fire = df2_wo_mostly_null_cols[df2_wo_mostly_null_cols['class_label'] == 0][col].mean() if -0.5 <= skewness <= 0.5 else df2_wo_mostly_null_cols[df2_wo_mostly_null_cols['class_label'] == 0][col].median()
    mean_or_median_fire = df2_wo_mostly_null_cols[df2_wo_mostly_null_cols['class_label'] == 1][col].mean() if -0.5 <= skewness <= 0.5 else df2_wo_mostly_null_cols[df2_wo_mostly_null_cols['class_label'] == 1][col].median()

    # Impute missing values based on class label
    for index, row in df2_wo_mostly_null_cols.iterrows():
        if pd.isnull(row[col]):
            if row['class_label'] == 0:
                df2_wo_mostly_null_cols.at[index, col] = mean_or_median_non_fire
            else:
                df2_wo_mostly_null_cols.at[index, col] = mean_or_median_fire

# Iterate over each categorical column
for col in categorical_cols_with_nans:

    mode_non_fire = df2_wo_mostly_null_cols[df2_wo_mostly_null_cols['class_label'] == 0][col].mode()[0]
    mode_fire = df2_wo_mostly_null_cols[df2_wo_mostly_null_cols['class_label'] == 1][col].mode()[0]

    # Impute missing values based on class label
    for index, row in df2_wo_mostly_null_cols.iterrows():
        if pd.isnull(row[col]):
            if row['class_label'] == 0:
                df2_wo_mostly_null_cols.at[index, col] = mode_non_fire
            else:
                df2_wo_mostly_null_cols.at[index, col] = mode_fire


#### Using only mean

In [ ]:
# Using mean only for the numerical data
# Using mode for the categorical data
# Calculating the values for each class independently i.e. if we use mean or mode we calculate it for non-fire nans using non-fire aggregation total, for fire nans we use fire aggergation totals.

numerical_cols = [
    'sumPrecipitation', 'dist_sumPrecipitation', 'maxDryBulbTemperature', 'dist_maxDryBulbTemperature',
    'minDryBulbTemperature', 'dist_minDryBulbTemperature', 'meanDryBulbTemperature', 'dist_meanDryBulbTemperature',
    'meanDewPointTemperature', 'dist_meanDewPointTemperature', 'meanWetBulbTemperature', 'dist_meanWetBulbTemperature',
    'meanWindSpeed', 'dist_meanWindSpeed', 'meanRelativeHumidity', 'dist_meanRelativeHumidity',
    'minRelativeHumidity', 'dist_minRelativeHumidity', 'maxRelativeHumidity', 'dist_maxRelativeHumidity',
    'maxWindSpeed', 'dist_maxWindSpeed', 'calculate_circular_meanWindDirection', 'dist_calculate_circular_meanWindDirection',
    'mode_functionWindDirection', 'dist_mode_functionWindDirection', 'dist_mode_functionSkyConditions', '10_sumPrecipitation', '10_maxDryBulbTemperature',
    '10_minDryBulbTemperature', '10_meanDryBulbTemperature', '10_meanDewPointTemperature', '10_meanWetBulbTemperature',
    '10_meanWindSpeed', '10_meanRelativeHumidity', '10_minRelativeHumidity', '10_maxRelativeHumidity',
    '10_maxWindSpeed', '10_calculate_circular_meanWindDirection', '10_mode_functionWindDirection',
    '30_sumPrecipitation', '30_maxDryBulbTemperature', '30_minDryBulbTemperature', '30_meanDryBulbTemperature',
    '30_meanDewPointTemperature', '30_meanWetBulbTemperature', '30_meanWindSpeed', '30_meanRelativeHumidity',
    '30_minRelativeHumidity', '30_maxRelativeHumidity', '30_maxWindSpeed', '30_calculate_circular_meanWindDirection',
    '30_mode_functionWindDirection', '60_sumPrecipitation', '60_maxDryBulbTemperature', '60_minDryBulbTemperature',
    '60_meanDryBulbTemperature', '60_meanDewPointTemperature', '60_meanWetBulbTemperature', '60_meanWindSpeed',
    '60_meanRelativeHumidity', '60_minRelativeHumidity', '60_maxRelativeHumidity', '60_maxWindSpeed',
    '60_calculate_circular_meanWindDirection', '60_mode_functionWindDirection', 'far_hist_avg_acres_burned', 'near_hist_avg_acres_burned'
]

categorical_cols_with_nans = [
    'mode_functionSkyConditions', '10_mode_functionSkyConditions', '30_mode_functionSkyConditions', '60_mode_functionSkyConditions'
]

# Imputing numerical columns with mean only
for col in numerical_cols:
    mean_non_fire = df3_wo_mostly_null_cols[df3_wo_mostly_null_cols['class_label'] == 0][col].mean()
    mean_fire = df3_wo_mostly_null_cols[df3_wo_mostly_null_cols['class_label'] == 1][col].mean()

    for index, row in df3_wo_mostly_null_cols.iterrows():
        if pd.isnull(row[col]):
            if row['class_label'] == 0:
                df3_wo_mostly_null_cols.at[index, col] = mean_non_fire
            else:
                df3_wo_mostly_null_cols.at[index, col] = mean_fire

# Imputing categorical columns with mode
for col in categorical_cols_with_nans:
    mode_non_fire = df3_wo_mostly_null_cols[df3_wo_mostly_null_cols['class_label'] == 0][col].mode()[0]
    mode_fire = df3_wo_mostly_null_cols[df3_wo_mostly_null_cols['class_label'] == 1][col].mode()[0]

    for index, row in df3_wo_mostly_null_cols.iterrows():
        if pd.isnull(row[col]):
            if row['class_label'] == 0:
                df3_wo_mostly_null_cols.at[index, col] = mode_non_fire
            else:
                df3_wo_mostly_null_cols.at[index, col] = mode_fire



### Checking nan counts after imputing the values

In [ ]:
pd.set_option('display.max_rows', None)
nan_counts_per_feature = df2_wo_mostly_null_cols.isnull().sum()
print("Counts of NaN Values per Feature:")
print(nan_counts_per_feature)
total_columns = len(df2_wo_mostly_null_cols.columns)
print("\nTotal Number of Columns:", total_columns)
pd.reset_option('display.max_rows')

Counts of NaN Values per Feature:
incident_name                                0
incident_created_year                        0
incident_created_month                       0
incident_created_day                         0
incident_created_hour                        0
incident_created_minute                      0
incident_latitude                            0
incident_longitude                           0
incident_geohash                             0
LCD_station_id                               0
LCD_station_elevation                        0
LCD_station_distance                         0
GHCN_station_id                              0
GHCN_station_elevation                       0
GHCN_station_distance                        0
sumPrecipitation                             0
dist_sumPrecipitation                        0
maxDryBulbTemperature                        0
dist_maxDryBulbTemperature                   0
minDryBulbTemperature                        0
dist_minDryBulbTemperature

In [ ]:
pd.set_option('display.max_rows', None)
nan_counts_per_feature = df3_wo_mostly_null_cols.isnull().sum()
print("Counts of NaN Values per Feature:")
print(nan_counts_per_feature)
total_columns = len(df3_wo_mostly_null_cols.columns)
print("\nTotal Number of Columns:", total_columns)
pd.reset_option('display.max_rows')

Counts of NaN Values per Feature:
incident_name                                0
incident_created_year                        0
incident_created_month                       0
incident_created_day                         0
incident_created_hour                        0
incident_created_minute                      0
incident_latitude                            0
incident_longitude                           0
incident_geohash                             0
LCD_station_id                               0
LCD_station_elevation                        0
LCD_station_distance                         0
GHCN_station_id                              0
GHCN_station_elevation                       0
GHCN_station_distance                        0
sumPrecipitation                             0
dist_sumPrecipitation                        0
maxDryBulbTemperature                        0
dist_maxDryBulbTemperature                   0
minDryBulbTemperature                        0
dist_minDryBulbTemperature

In [ ]:
print_min_max_mean_values(df2_wo_mostly_null_cols, features)

Features Within Acceptable Range:
sumPrecipitation                    PhyMin = 0          PhyMax = inf        ✓ Min = 0.00000    ✓ Max = 5.49000    ✓ Mean = 0.01976   
maxDryBulbTemperature               PhyMin = -58        PhyMax = 122        ✓ Min = 27.00000   ✓ Max = 121.00000  ✓ Mean = 72.86246  
minDryBulbTemperature               PhyMin = -58        PhyMax = 122        ✓ Min = 1.00000    ✓ Max = 98.00000   ✓ Mean = 58.56791  
meanDryBulbTemperature              PhyMin = -58        PhyMax = 122        ✓ Min = 8.77358    ✓ Max = 109.22917  ✓ Mean = 64.93115  
meanDewPointTemperature             PhyMin = -22        PhyMax = 86         ✓ Min = -18.64286  ✓ Max = 76.16667   ✓ Mean = 41.84513  
meanWetBulbTemperature              PhyMin = -22        PhyMax = 86         ✓ Min = 8.15094    ✓ Max = 82.00000   ✓ Mean = 52.79267  
meanWindSpeed                       PhyMin = 0          PhyMax = 200        ✓ Min = 0.00000    ✓ Max = 32.45714   ✓ Mean = 6.53995   
meanRelativeHumidity        

In [ ]:
print_min_max_mean_values(df3_wo_mostly_null_cols, features)

Features Within Acceptable Range:
sumPrecipitation                    PhyMin = 0          PhyMax = inf        ✓ Min = 0.00000    ✓ Max = 5.49000    ✓ Mean = 0.01979   
maxDryBulbTemperature               PhyMin = -58        PhyMax = 122        ✓ Min = 27.00000   ✓ Max = 121.00000  ✓ Mean = 72.86331  
minDryBulbTemperature               PhyMin = -58        PhyMax = 122        ✓ Min = 1.00000    ✓ Max = 98.00000   ✓ Mean = 58.56740  
meanDryBulbTemperature              PhyMin = -58        PhyMax = 122        ✓ Min = 8.77358    ✓ Max = 109.22917  ✓ Mean = 64.93115  
meanDewPointTemperature             PhyMin = -22        PhyMax = 86         ✓ Min = -18.64286  ✓ Max = 76.16667   ✓ Mean = 41.66159  
meanWetBulbTemperature              PhyMin = -22        PhyMax = 86         ✓ Min = 8.15094    ✓ Max = 82.00000   ✓ Mean = 52.69865  
meanWindSpeed                       PhyMin = 0          PhyMax = 200        ✓ Min = 0.00000    ✓ Max = 32.45714   ✓ Mean = 6.54017   
meanRelativeHumidity        

## Exporting the dataframes

In [ ]:
base_path = "/content/drive/My Drive/ML6140  - Project/Data/"
df1_wo_nan_rows_cols.to_csv(base_path + 'enhanced_data_df1.csv', index=False)
df2_wo_mostly_null_cols.to_csv(base_path + 'enhanced_data_df2.csv', index=False)
df3_wo_mostly_null_cols.to_csv(base_path + 'enhanced_data_df3.csv', index=False)
